### Imports

In [32]:
import openai
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel

_ = load_dotenv(find_dotenv())

client = openai.Client()

### Criando o Assistant

In [2]:
vector_store = client.beta.vector_stores.create(name = 'Pedidos Stoky')

In [5]:
files = ['arquivos/pedido_renner.pdf',
         'arquivos/pedido_fertak.pdf',
         'arquivos/pedido_iquine.pdf']
file_stream = [open(f, 'rb') for f in files]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=file_stream
)

In [6]:
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=3, failed=0, in_progress=0, total=3)


In [7]:
assitant = client.beta.assistants.create(
    name="Conversor de pedidos",
    instructions="Você é um conversor de pedidos que estão no formato pdf e serão convertidos para xlsx. Você deve ler o arquivo pdf informado e retornar um arquivo xlsx com os dados dos \
                 itens que estão nele geralmente como uma tabela e ler os campos dessa tabela que geralmente possuem colunas com nomes como Código, Descrição, Valor unitário, quantidade entre \
                 outros campos e retornar em uma planilha do excel no formato xlsx os mesmos dados que estão no pdf. Caso não consiga deve informar que não foi possível realizar a conversão.",
    tools=[{'type': 'file_search'}],
    tool_resources={'file_search': {'vector_store_ids': [vector_store.id]}},
    model='gpt-4o'
)

### Atualizando o Assistant

In [33]:
class Item(BaseModel):
    codigo: int
    descricao: str
    quantidade: float
    valor_unitario: float
    valor_total: float

class Pedido(BaseModel):
    numero_pedido: str
    items: list[Item]

response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "get_pedido",
        "description": "lista de itens com os seus dados",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "referencia": {
                    "type": "string",
                    "description": "É o ID que identifica o item também chamado de código, produto, item, etc."
                },
                "descricao": {
                    "type": "string",
                    "description": "É a descrição do item."
                },
                "qtdade": {
                    "type": "number",
                    "description": "É a quantidade do item"
                },
                "vlr_unit": {
                    "type": "number",
                    "description": "É a valor unitário do item"
                },
                "vlr_total": {
                    "type": "number",
                    "description": "É a valor total do item"
                }
            },
            "required": ["referencia", "descricao", "qtdade", "vlr_unit", "vlr_total"],
            "additionalProperties": False,
        }
    }
}

In [35]:
assitant = client.beta.assistants.update(
    assistant_id='asst_iVKgnw03y958jwbzN8URbBEj',
    instructions="Você é um conversor de pedidos que estão no formato pdf e serão convertidos para json. Você deve ler o arquivo pdf informado e retornar um arquivo json com os dados dos \
                 itens que estão nele geralmente como uma tabela e ler os campos dessa tabela que geralmente possuem colunas com nomes como Código, Descrição, Valor unitário, quantidade entre \
                 outros campos e retornar em um objeto json os mesmos dados que estão no pdf. Caso não consiga deve informar que não foi possível realizar a conversão.",
    response_format=Pedido,
    model='gpt-4o-2024-08-06'
)

TypeError: Object of type ModelMetaclass is not JSON serializable

### Criar uma Thread

In [8]:
thread = client.beta.threads.create()

### Adiciona a mensagem na Thread criada

In [9]:
mensagem_texto = 'Segundo o documento fornecido pedido iquine me retorne os dados dos itens no formato xlsx.'


message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content=mensagem_texto
)

### Rode a Thread no Assistant

In [10]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assitant.id,
    instructions='O nome do usuário é Alisson e ele é um usuário Premium.'
)

### Aguarde a Thread finalizar

In [11]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )
print(run.status)

completed


### Verifique a resposta

In [12]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print('Errro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_WFukwByyUdAbgKRiCXCHD2yq', assistant_id='asst_iVKgnw03y958jwbzN8URbBEj', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Baseado no documento fornecido, segue uma tabela com os itens extraídos no formato CSV:\n\n```plaintext\nNº,Produto,Descrição,Emb.,Qtde,Saldo,Preço Líq.,Preço total,% IPI,Preço total + IPI\n1,50300201,DIATEX PINTA MAIS BRANCO NEVE 3,6L,4,48,48,24,59,1.180,32,0,00,1.180,32\n2,11100104,SELADORA CONCENTRADA INCOLOR 900 ML,6,60,60,16,51,990,60,3,25,1.022,79\n3,11100101,SELADORA CONCENTRADA INCOLOR 3,6L,4,12,12,59,10,709,20,3,25,732,25\n4,20100101,VERNIZ COPAL BRILHANTE 3,6L,4,60,60,50,28,3.016,80,3,25,3.114,85\n5,20100104,VERNIZ COPAL BRILHANTE 900 ML,6,96,96,15,54,1.491,84,3,25,1.540,32\n6,19105501,VERNIZ EXTRARRAPIDO IMBUIA 3,6L,4,72,72,55,49,3.995,28,3,25,4.125,13\n7,19109101,VERNIZ EXTRARRAPIDO MOGNO 3,6L,4,36,36,63,62,2.290,32,3,25,2.364,76\n8,19105504,VERNIZ EXTRARRAPIDO IMB

In [14]:
mensagens.data[0].content[0].text.value

'Baseado no documento fornecido, segue uma tabela com os itens extraídos no formato CSV:\n\n```plaintext\nNº,Produto,Descrição,Emb.,Qtde,Saldo,Preço Líq.,Preço total,% IPI,Preço total + IPI\n1,50300201,DIATEX PINTA MAIS BRANCO NEVE 3,6L,4,48,48,24,59,1.180,32,0,00,1.180,32\n2,11100104,SELADORA CONCENTRADA INCOLOR 900 ML,6,60,60,16,51,990,60,3,25,1.022,79\n3,11100101,SELADORA CONCENTRADA INCOLOR 3,6L,4,12,12,59,10,709,20,3,25,732,25\n4,20100101,VERNIZ COPAL BRILHANTE 3,6L,4,60,60,50,28,3.016,80,3,25,3.114,85\n5,20100104,VERNIZ COPAL BRILHANTE 900 ML,6,96,96,15,54,1.491,84,3,25,1.540,32\n6,19105501,VERNIZ EXTRARRAPIDO IMBUIA 3,6L,4,72,72,55,49,3.995,28,3,25,4.125,13\n7,19109101,VERNIZ EXTRARRAPIDO MOGNO 3,6L,4,36,36,63,62,2.290,32,3,25,2.364,76\n8,19105504,VERNIZ EXTRARRAPIDO IMBUIA 900ML,6,60,60,18,18,1.090,80,3,25,1.126,25\n9,19109104,VERNIZ EXTRARRAPIDO MOGNO 900ML,6,120,120,20,15,2.418,00,3,25,2.496,58\n10,70300205,DELANIL RENDE MUITO BRANCO NEVE 18L,1,30,30,155,00,4.650,00,0,00,4.65